# Prepare the SGLang SageMaker container

❗This notebook works well on `ml.g5.xlarge` instance with 50GB of disk size and `PyTorch 2.2.0 Python 3.10 CPU optimized kernel` from **SageMaker Studio Classic** or `Python3 kernel` from **JupyterLab**.

This notebook has been rewritten based on [sagemaker-genai-hosting-examples/Deepseek/SGLang-Deepseek/deepseek-r1-llama-70b-sglang.ipynb](https://github.com/aws-samples/sagemaker-genai-hosting-examples/blob/main/Deepseek/SGLang-Deepseek/deepseek-r1-llama-70b-sglang.ipynb)

Note that SageMaker provides [pre-built SageMaker AI Docker images](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html) that can help you quickly start with the model inference on SageMaker. It also allows you to [bring your own Docker container](https://docs.aws.amazon.com/sagemaker/latest/dg/adapt-inference-container.html) and use it inside SageMaker AI for training and inference. To be compatible with SageMaker AI, your container must have the following characteristics:

- Your container must have a web server listening on port `8080`.
- Your container must accept POST requests to the `/invocations` and `/ping` real-time endpoints.

In this notebook, we'll make a custom Docker container to adapt the [SGLang](https://github.com/sgl-project/sglang) framework to run on SageMaker AI endpoints. SGLang is a serving framework for large language models that provides state-of-the-art performance, including a fast backend runtime for efficient serving with RadixAttention, extensive model support, and an active open-source community. For more information refer to [https://docs.sglang.ai/index.html](https://docs.sglang.ai/index.html) and [https://github.com/sgl-project/sglang](https://github.com/sgl-project/sglang).

By using the custom Docker container for SGLang, you can run advanced AI models like the [DeepSeek-R1-Distill-Llama-8B](https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Llama-8B) on a SageMaker AI endpoint.


### Set up Environment

In [ ]:
%%capture --no-stderr

!pip install -U "sagemaker>=2.237.1"
!pip install -U sagemaker-studio-image-build==0.6.0

In [ ]:
!pip list | grep -E -w "sagemaker|sagemaker_studio_image_build"

### Prepare the SGLang SageMaker container

In [ ]:
DOCKER_IMAGE = "sglang-sagemaker"
DOCKER_IMAGE_TAG = "latest"

[sm-docker](https://github.com/aws-samples/sagemaker-studio-image-build-cli) is a CLI for building Docker images in SageMaker Studio using AWS CodeBuild

In [ ]:
%%time
!cd ./container && sm-docker build . --repository {DOCKER_IMAGE}:{DOCKER_IMAGE_TAG} --build-arg BASE_IMAGE='lmsysorg/sglang:v0.4.4.post1-cu125'

In [ ]:
from sagemaker.session import Session

session = Session()
region = session._region_name

ecr_uri = f'{session.account_id()}.dkr.ecr.{region}.amazonaws.com/{DOCKER_IMAGE}:{DOCKER_IMAGE_TAG}'
ecr_uri

### References

- [sm-docker](https://github.com/aws-samples/sagemaker-studio-image-build-cli): a CLI for building Docker images in SageMaker Studio using AWS CodeBuild
- [sagemaker-genai-hosting-examples/Deepseek/SGLang-Deepseek/deepseek-r1-llama-70b-sglang.ipynb](https://github.com/aws-samples/sagemaker-genai-hosting-examples/blob/main/Deepseek/SGLang-Deepseek/deepseek-r1-llama-70b-sglang.ipynb)